In [36]:
import polars as pl
from spatial_filtering import arrays, constants, direction_of_arrival
import importlib

importlib.reload(arrays)
importlib.reload(constants)
importlib.reload(direction_of_arrival)

meerkat_pos = pl.read_excel('meerkat positions.xlsx')
meerkat_pos = meerkat_pos.with_columns(distance_from_center_m=(pl.col('East')**2 + pl.col('North')**2 + pl.col('Up')**2).sqrt())
meerkat_pos_core = meerkat_pos.filter(pl.col('distance_from_center_m') < 750)
core_indices = [int(r[-2:]) for r in meerkat_pos_core['Antenna'].to_list()]
data_np = meerkat_pos_core[['East', 'North', 'Up']].to_numpy()

from spatial_filtering.arrays import Array
import numpy as np

array = arrays.Array(data_np)
N = array.num_antennas
f = 1.2276e9              # Frequency 1.2276 GHz GPS L2 band.
wavelength = constants.c / f
snapshots = 2000


In [2]:
from dadanalyse import read_dada
GPS_CHANNEL = 50
POL = 1
# J1644-4559_2024-02-16T11:21:22.092_29_73532270706688.dada
dada = read_dada("gps.dada")
print(dada.get_header())
dada = dada.combined_data()



{'HEADER': 'DADA', 'HDR_VERSION': '1.0', 'HDR_SIZE': 4096, 'DADA_VERSION': '1.0', 'OBS_ID': 'None', 'FILE_SIZE': '3048214528', 'FILE_NUMBER': '0', 'UTC_START': '1708082482.092702505', 'MJD_START': '60356.47317236924195', 'OBS_OFFSET': '0', 'OBS_OVERLAP': '0', 'SOURCE': 'J1644-4559_Offset1', 'RA': '16:44:26.25', 'DEC': '-45:59:09.6', 'TELESCOPE': 'MeerKAT', 'INSTRUMENT': 'CBF-Feng', 'RECEIVER': 'L-band', 'FREQ': '1284000000.000000', 'BW': '856000000.000000', 'TSAMP': '4.7850467290', 'BYTES_PER_SECOND': '3424000000.000000', 'NBIT': 8, 'NDIM': 2, 'NPOL': 2, 'NCHAN': 64, 'NANT': 57, 'ORDER': 'TAFTP', 'INNER_T': 256, 'SYNC_TIME': '1708039531.000000', 'SAMPLE_CLOCK': '1712000000.000000', 'SAMPLE_CLOCK_START': '73532270706688', 'CHAN0_IDX': 1728, 'Lowest_freq (MHz)': 1217.125}


In [8]:
dada.shape

(208896, 57, 64, 2)

In [37]:
X = dada[0:2000, core_indices, GPS_CHANNEL, POL].T

In [38]:
from skyfield.api import load, Topos, EarthSatellite
from datetime import datetime
import numpy as np

# Location: MeerKAT Telescope (converted to decimal degrees)
lat = -30.71106
lon = 21.44389
elevation_m = 1086.6

# Time: Convert UNIX timestamp to UTC
timestamp = 1708082482.092702505
dt = datetime.utcfromtimestamp(timestamp)
dt

datetime.datetime(2024, 2, 16, 11, 21, 22, 92702)

In [39]:
R = X @ X.conj().T / N

In [40]:
import numpy as np

evals, evecs = np.linalg.eigh(R)

In [41]:
import matplotlib.pyplot as plt

plt.plot(evals)
plt.show()

In [32]:
evals[-15:]

array([ 24490.18 ,  24926.332,  25154.76 ,  25361.377,  25918.19 ,
        26566.24 ,  27081.904,  27656.371,  28827.396,  30058.643,
        32373.867,  37933.08 ,  71261.484,  93760.695, 198970.06 ],
      dtype=float32)

In [ ]:


theta_steps = 10_000
phi_steps = 10_000

output = direction_of_arrival.MUSICDOA2D().get_directions(
    array,
    R, 
    num_interferers=5,
    wavelength = wavelength,
    theta_min_deg = -90,
    theta_max_deg = 90,
    theta_steps=theta_steps,
    phi_min_deg = -180,
    phi_max_deg = 180,
    phi_steps=phi_steps,
)

In [ ]:

import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'iframe'  # or 'iframe' or 'colab' depending on your environment

# Filter top 1% Q values (adjust as needed)
threshold = output['Q'].quantile(0.999975)
high_Q = output.filter(pl.col('Q') >= threshold)

# Create 3D scatter plot
fig = px.scatter_3d(
    high_Q,
    x='theta',  # azimuth
    y='r',      # range
    z='Q',      # MUSIC power
    color='Q',
    color_continuous_scale='Jet',
    opacity=0.8
)

# Update layout for better visuals
fig.update_layout(
    title='High-Q MUSIC Spectrum Peaks',
    scene=dict(
        xaxis_title='Angle (degrees)',
        yaxis_title='Range (meters)',
        zaxis_title='Q (dB)'
    )
)

fig.show()

The MJD I want is 60356.47317236924195




In [64]:
from skyfield.api import load, EarthSatellite
from skyfield.iokit import parse_tle_file
import json
ts = load.timescale()

with load.open('gp2_2.tle') as f:
    data = json.load(f)

ts = load.timescale()
sats = [EarthSatellite.from_omm(ts, fields) for fields in data]

print('Loaded', len(data), 'satellites')

Loaded 54 satellites


In [73]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
import astropy.units as u

# Observer location (MeerKAT)
location = EarthLocation(lat=-30.71106*u.deg,
                         lon=21.44389*u.deg,
                         height=1086.6*u.m)

# Time of observation (UTC)
obstime = Time('2024-02-16 11:21:22')

# RA/DEC coordinates (example)
ra_str = '16:44:26.25'
dec_str = '-45:59:09.6'

# Create SkyCoord object in ICRS (equatorial)
sky_coord = SkyCoord(ra=ra, dec=dec, frame='icrs')

# Create AltAz frame at observer location and time
altaz_frame = AltAz(obstime=obstime, location=location)

# Transform RA/DEC to AltAz
altaz = sky_coord.transform_to(altaz_frame)

# Output altitude and azimuth
print(f"Altitude: {altaz.alt:.2f}")
print(f"Azimuth: {altaz.az:.2f}")

Altitude: 24.02 deg
Azimuth: 229.33 deg


In [74]:
from skyfield.api import wgs84, Topos

meerkat = Topos(latitude_degrees=-30.71106,
                longitude_degrees=21.44389,
                elevation_m= 1086.6)

obs_site =  meerkat
t0 = ts.utc(2024, 2, 15)
t1 = ts.utc(2024, 2, 17)
t_obs = ts.utc(2024, 2, 16, 11, 21, 22)
sats_visible = []
sats_processed = set()

for satellite in sats:
    # There are some duplicate TLEs - don't worry about this.
    if satellite.name in sats_processed:
        continue
    event_dict = {}
    event_dict['name'] = satellite.name

    topocentric = (satellite - obs_site).at(t_obs)
    alt, az, _ = topocentric.altaz()
    event_dict['alt'] = alt
    event_dict['az'] = az
    if alt.degrees > 0:
        sats_visible.append(event_dict)
    sats_processed.add(satellite.name)

df = pl.from_records(sats_visible)
#df.write_excel('sats.xlsx')
#len(sats_processed)
len(sats_visible)

10

In [60]:
from astropy.time import Time

# Your MJD value (example)
mjd = 60356.47317236924195

# Create Time object
t = Time(mjd, format='mjd', scale='utc')

# Print UTC as ISO format
print("UTC Time:", t.utc.iso)

# Or get components
print("Year:", t.utc.datetime.year)
print("Datetime:", t.utc.datetime)

UTC Time: 2024-02-16 11:21:22.093
Year: 2024
Datetime: 2024-02-16 11:21:22.092702


np.float64(44.369909028490135)